### Comitee Machine para categorização de produtos 3p em categorias 1p
Recebe como entrada as predições de diversos modelos de catedorização de produtos 3p em categorias 1p e faz uma votação para escolher a classe final selecionada.
- Quando todos os modelos escolhem categorias diferentes, usa as predições resultantes da associação entre as categorais 3p e 1p (sdf_category_similarity - associação checada manualmente).
-  Como critério de desempate, usa a probabilidade de classe mais alta.

In [0]:
import pandas as pd
from pyspark.sql import functions as F, Window, DataFrame
from pyspark.sql.types import *
from typing import List, Union
from collections import Counter
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [0]:
# Predições Modelo de Frequência
sdf_fr_pred = spark.read.parquet(f'/mnt/advisor-hml/data/07_model_output/3p_categorization/M1_freq')

# Predições Modelo Random Forest
sdf_rf_pred = spark.read.parquet(f'/mnt/advisor-hml/data/07_model_output/3p_categorization/RF')

# Predições Modelo Random Forest
sdf_lr_pred = spark.read.parquet(f'/mnt/advisor-hml/data/07_model_output/3p_categorization/LR')

# Associação caterorias 3p - 1p 
sdf_category_similarity = (
    spark.read
    .option("mergeSchema", True)
    .option("encoding", "UTF-8")
    .csv(f'/mnt/advisor-hml/data/07_model_output/3p_categorization/3p_1p_category_similarity/3p_1p_category_similarity.csv', sep=";", header=True)
)

#Dados rotulados filtrados 
sdf_labels = (
    spark.read
    .option("mergeSchema", True)
    .option("encoding", "UTF-8")
    .csv(f'/mnt/advisor-hml/data/07_model_output/3p_categorization/3p_categorization_labels/labeled_data.csv', sep=";", header=True)
).select('SKU', 'Rank', 'MartinsCategory')

#Dados rotulados completo
sdf_labels_extra = (
    spark.read
    .option("mergeSchema", True)
    .option("encoding", "UTF-8")
    .csv(f'/mnt/advisor-hml/data/07_model_output/3p_categorization/3p_categorization_labels/labeled_data_extra.csv', sep=";", header=True)
).select('SKU', 'Rank', 'MartinsCategory')

In [0]:
sdf_comitee_pred = sdf_lr_pred.join(sdf_rf_pred.select('SKU', 'ProductCategoryRF', 'ProbabilityRF'), ['SKU'], how='left')
sdf_comitee_pred = sdf_comitee_pred.join(sdf_fr_pred.select('SKU', 'ProductCategoryFR1', 'ProbabilityFR1'), ['SKU'], how='left')
sdf_comitee_pred = sdf_comitee_pred.join(sdf_category_similarity, ['OriginalCategory'] , how='left')

In [0]:
models_number = 4

@udf(returnType=StringType())
def voting_category(rf, lg, freq, sim, p_rf, p_lr, p_freq):
  '''
  Regras para associação dos modelos do comitê (votação e desempate)
  rf, lg, freq, sim = String com o nome das colunas contendo predições respectivamnete dos modelos random forest, logistic regression, frequência e similaridade
  p_rf, p_lr, p_freq = String com o nome das colunas contendo as probabilidades dos modelos random forest, logistic regression, frequência
  '''
  
  predicoes = [rf, lg, freq, sim]
  prob = [p_rf, p_lr, p_freq]
  
  # Lista as diferentes categorias preditas pelos modelos para um mesmo sku (maior o tamanho dessa lista, maior a discordância entre os modelos)
  distinct_class = []
  for x in predicoes:
    if x not in distinct_class:
        distinct_class.append(x)
 
  #Se os modelos retornarem classes distintas para o produtos, usar as predições do mapeamento de similaridade das categorias 3p e 1p
  if (len(distinct_class) == models_number):
    
    if sim is None: 
      valor = 'NAO ENCONTRADO' # a não ser que não haja resposta  por similaridade de categorias
    else:
      valor = sim
     
    #Se houver empate 2x2, eescolher o resultado do modelo com maior probabilidade 
    if (len(distinct_class) == models_number/2):  
      try:
        k, max_prob = 0, 0
        for pi in prob:
            if(max_prob < pi):
                max_prob = pi
                valor = classes[k]
            k += 1
      except:
        pass
      
  #Caso não ocorram as excessões acima, seleciona a categoria que tiver o maior número de votos   
  else:
    occurence_count = Counter(predicoes)
    valor = occurence_count.most_common(1)[0][0]
  
  return valor


nomes_colunas = ['ProductCategoryLR','ProductCategoryRF','ProductCategoryFR1','ProductCategorySim','ProbabilityLR','ProbabilityRF','ProbabilityFR1']

sdf_comitee_pred_final = (
  sdf_comitee_pred
  .withColumn(
    'SelectedCategory',
    voting_category(*nomes_colunas,)
  )
) 

#sdf_comitee_pred_final.select('OriginalCategory','ProductCategoryLR','ProductCategoryRF','ProductCategoryFR1','ProductCategorySim','SelectedCategory','ProductDescription').display()

sdf_comitee_selected_category = sdf_comitee_pred_final.select('SKU', 'ProductDescription', 'OriginalCategory','SelectedCategory')

In [0]:
sdf_comitee_selected_category.display()

SKU,ProductDescription,OriginalCategory,SelectedCategory
alhoforte_171,Molho de Alho 145 ml,condimentos-e-temperos,TEMPERO INDUSTRIALIZADO
alimentosphinus_2001004,"Doce de Cocada 600g Zero Adicao de Acucares, 58% frutas frescas - PHINUS ( Caixa Display com 24 uni",doces,GOMA DE MASCAR/DROPS E CARAMELOS
alimentosphinus_2001007,"Doce de Pe de Moleque 600g Zero Adicao de Acucares, 55% Amendoim - PHINUS ( Caixa Display com 24 un",doces,GOMA DE MASCAR/DROPS E CARAMELOS
alimentosphinus_2002039,Bombom 72% Cacau 270g Zero Adicao de Acucares - PHINUS ( Caixa Display com 18 unidades de 15g),doces,GOMA DE MASCAR/DROPS E CARAMELOS
flamboyant_3823,Goiabada Tablete Flamboyant 24x400g,doces,DOCE
flormel_7896653703169,"Doce de Leite Cremoso Flormel, Zero Acucar - Pote com 210g",doces,DOCE
flormel_7896653703282,"Doce de Abacaxi com Coco Flormel, Zero acucar - Caixa com 24 Unidades de 20g cada",doces,GOMA DE MASCAR/DROPS E CARAMELOS
gama_24088,Geleia Linea Uva 230g,geleias,DOCE
gamaba_23886,Geleia Linea Frutas Vermelhas 230g,geleias,NAO ENCONTRADO
gamarj_23886,Geleia Linea Frutas Vermelhas 230g,geleias,NAO ENCONTRADO


In [0]:
sdf_comitee_selected_category \
    .write.mode('overwrite') \
    .format('parquet') \
    .save('/mnt/analyticsquadmkt/data-analytics-projects/CATEGORIZATION 3P/OUTPUT/categorization_3p/')

### Avaliação de desempenho do modelo

In [0]:
print('Quantidade de produtos 3p com categoria 1p não encontrada:', sdf_comitee_pred_final.where(F.col('SelectedCategory')=='NAO ENCONTRADO').count())
print()
print('Contagem de produtos não categorizados encontrados por categoria 3p')
sdf_comitee_pred_final.where(F.col('SelectedCategory')=='NAO ENCONTRADO').groupBy('OriginalCategory').count().orderBy(F.col('count').desc()).display()

Quantidade de produtos 3p com categoria 1p não encontrada: 11983

Contagem de produtos não categorizados encontrados por categoria 3p

OriginalCategory,count
porta-de-aluminio,1071
equipamentos-medicos-hospitalares,973
corte-bovino,925
janela-de-aluminio,847
descartaveis,678
medicamentos,540
primeiros-socorros,398
acessorios-musicais,395
corte-suino,339
acessorios-esportivos,321


In [0]:
sdf_model_evaluation_comitee = sdf_labels.join(sdf_comitee_pred_final, ['SKU'], how='inner')
#sdf_model_evaluation_comitee.select('ProductDescription','OriginalCategory','ProductCategoryLR','ProductCategoryRF','ProductCategoryFR1','ProductCategorySim','SelectedCategory','MartinsCategory').display()

y = sdf_model_evaluation_comitee.select('MartinsCategory').toPandas()

ycalc = sdf_model_evaluation_comitee.select('SelectedCategory').toPandas()

print('Accuracy:', accuracy_score(y, ycalc))
print('F1_score:', f1_score(y, ycalc, average='weighted'))
print('Precision:', precision_score(y, ycalc, average='weighted'))
print('Recall:', recall_score(y, ycalc, average='weighted'))

Accuracy: 0.9333333333333333
F1_score: 0.9379756660901417
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
Precision: 0.9555742296918768
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
Recall: 0.9333333333333333

In [0]:
sdf_model_evaluation_comitee_extra = sdf_labels_extra.join(sdf_comitee_pred_final, ['SKU'], how='inner').orderBy('Rank')

#sdf_model_evaluation_comitee_extra.select('ProductDescription','OriginalCategory','ProductCategoryLR','ProductCategoryRF','ProductCategoryFR1','ProductCategorySim','SelectedCategory','MartinsCategory').display()

y = sdf_model_evaluation_comitee_extra.select('MartinsCategory').toPandas()

ycalc = sdf_model_evaluation_comitee_extra.select('SelectedCategory').toPandas()

print('Accuracy:', accuracy_score(y, ycalc))
print('F1_score:', f1_score(y, ycalc, average='weighted'))
print('Precision:', precision_score(y, ycalc, average='weighted'))
print('Recall:', recall_score(y, ycalc, average='weighted'))

Accuracy: 0.8817567567567568
F1_score: 0.8983670245439956
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
Precision: 0.9317114192114193
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
Recall: 0.8817567567567568